# **Facial Recognition System: Siamese Network Implementation**

This notebook implements a facial verification system using Siamese Neural Networks. The system can determine whether two face images belong to the same person.

## Key Components:
1. **Data Collection**: Webcam-based face image capture
2. **Data Processing**: Creating image pairs for training
3. **Model Architecture**: Siamese network with shared weights
4. **Training**: Using triplet loss to learn facial embeddings
5. **Evaluation**: Testing the model's verification accuracy
6. **Deployment**: Saving the model for real-time applications


# **Setup and dependencies**

In [ ]:
#%pip install tensorflow==2.15.1 opencv-python matplotlib

In [ ]:
import tensorflow as tf
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import uuid
import time

In [ ]:
#Import tensor flow dependencies - Functional API

from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten, BatchNormalization


#Model: Lets you build and train a neural network.
#Layer: The base class for all Keras layers.
#Conv2D: Adds convolutional layers to extract image features.
#Dense: Fully connected layer for making decisions.
#MaxPooling2D: Reduces image size while keeping key features.
#Input: Defines the input shape of the model.
#Flatten: Turns multi-dimensional data into a flat vector.


GPU Setup for faster training

In [ ]:
#enable memory growth for GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

#for multiple gpus
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

#  Data collection

**Collecting Face Images**

The system includes a webcam-based data collection script that:

1. Captures images from the webcam
2. Detects faces using Haar Cascade
3. Crops and saves images to appropriate directories (anchor/positive)

**Key Terms:**
- **Anchor Images**: Reference images of the target person
- **Positive Images**: Different images of the same person
- **Negative Images**: Images of different people

**Create folder structures**

In [ ]:
#Setting paths for directories

POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

 **Collect positive and anchor images using haar cascade model**

In [ ]:
os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))

## Webcam Face Capture Implementation

This section implements a face detection and capture system with the following features:
- Real-time face detection using Haar Cascade
- Automatic and manual image capture modes
- Intelligent cropping to focus on detected faces
- Progress tracking for captured images

**Usage Instructions:**
- Press 'a' to capture an anchor image
- Press 'p' to capture a positive image
- Press 'c' to toggle automatic capture mode
- Press 'q' to quit

In [ ]:
import cv2
import os
import uuid
import time
import numpy as np


# create directories if they don't exist
for directory in [POS_PATH, NEG_PATH, ANC_PATH]:
    os.makedirs(directory, exist_ok=True)

# Establish a connection to the webcam
cap = cv2.VideoCapture(1)  

# Getting the maximum resolution from  camera
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print(f"Camera resolution: {width}x{height}")

# Trying to set a higher resolution 
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# verifying the new resolution
new_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
new_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print(f"New camera resolution: {new_width}x{new_height}")

# Loading face detection cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Variables for automatic capture
auto_capture = False  
last_capture_time = 0
capture_interval = 0.5  # Time between captures
save_as_anchor = True  # Toggle between anchor and positive
num_captured = {'anchor': 0, 'positive': 0}  # Counter for captured images
max_images = 300  # number of images to capture per class

def save_image(frame, save_type):
    global num_captured
    
    if save_type == 'anchor':
        path = ANC_PATH
        num_captured['anchor'] += 1
    else:
        path = POS_PATH
        num_captured['positive'] += 1
        
    imgname = os.path.join(path, f'{uuid.uuid1()}.jpg')
    cv2.imwrite(imgname, frame)
    print(f"Saved {save_type}: {imgname}, size: {frame.shape[:2]}, Total: {num_captured[save_type]}")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break
    
    #vreate a copy for display purposes
    display_frame = frame.copy()
    
    #face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    #status text to show 
    status_text = f"Auto-capture: {'ON' if auto_capture else 'OFF'} | "
    status_text += f"A: {num_captured['anchor']}, P: {num_captured['positive']}"
    if auto_capture:
        status_text += f" | Next: {'Anchor' if save_as_anchor else 'Positive'}"
    
    if len(faces) > 0:
        # find the largest face
        face = max(faces, key=lambda rect: rect[2] * rect[3])
        x, y, w, h = face
        
        # make the crop square and a bit larger than the detected face
        center_x, center_y = x + w//2, y + h//2
        size = max(w, h) * 1.5  # 50% larger than the face
        size = min(size, new_width, new_height)  # to make sure we dont pass the boundaries
        
        # Calculate crop coordinates
        x1 = max(0, int(center_x - size//2))
        y1 = max(0, int(center_y - size//2))
        x2 = min(int(new_width), int(x1 + size))
        y2 = min(int(new_height), int(y1 + size))
        
        # Draw rectangle for visualization
        cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Prepare the cropped frame
        cropped_frame = frame[y1:y2, x1:x2]
        
        # Automatic capture logic
        current_time = time.time()
        if auto_capture and (current_time - last_capture_time) > capture_interval:
            # Check if we've reached the maximum number for either class
            if (save_as_anchor and num_captured['anchor'] < max_images) or \
               (not save_as_anchor and num_captured['positive'] < max_images):
                
                # Save the image
                save_type = 'anchor' if save_as_anchor else 'positive'
                save_image(cropped_frame, save_type)
                
                # Toggle between anchor and positive for next capture
                save_as_anchor = not save_as_anchor
                
                # Update last capture time
                last_capture_time = current_time
                
            # Check if we've captured enough of both types
            elif num_captured['anchor'] >= max_images and num_captured['positive'] >= max_images:
                print(f"Maximum images captured: {max_images} anchors and {max_images} positives")
                auto_capture = False
    else:
        # If no face detected, use center crop (simpler fallback)
        center_x, center_y = int(new_width)//2, int(new_height)//2
        size = min(400, int(new_width), int(new_height))  # 400×400 crop or smaller if needed
        
        # Calculate crop coordinates
        x1 = max(0, center_x - size//2)
        y1 = max(0, center_y - size//2)
        x2 = min(int(new_width), x1 + size)
        y2 = min(int(new_height), y1 + size)
        
        # Draw rectangle for visualization
        cv2.rectangle(display_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        
        # Prepare the cropped frame
        cropped_frame = frame[y1:y2, x1:x2]
    
    # Display status information on screen
    cv2.putText(display_frame, status_text, (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    
    # Add usage instructions
    instructions = "a: manual anchor | p: manual positive | c: toggle auto-capture | q: quit"
    cv2.putText(display_frame, instructions, (10, int(new_height) - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # Show both the full frame with rectangle and the cropped version
    cv2.imshow('Full Frame', display_frame)
    
    # Only show the cropped frame if it exists
    if cropped_frame is not None and cropped_frame.size > 0:
        cv2.imshow('Cropped', cropped_frame)
    
    # Handle keyboard input
    key = cv2.waitKey(1) & 0xFF
    
    # Manual capture mode
    if key == ord('a'):
        if len(faces) > 0:  # Only save if a face is detected
            save_image(cropped_frame, 'anchor')
    
    elif key == ord('p'):
        if len(faces) > 0:  # Only save if a face is detected
            save_image(cropped_frame, 'positive')
    
    # Toggle automatic capture
    elif key == ord('c'):
        auto_capture = not auto_capture
        print(f"Auto-capture: {'ON' if auto_capture else 'OFF'}")
        if auto_capture:
            last_capture_time = time.time()  # Reset timer
    
    # Quit
    elif key == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

print(f"Capture session complete:")
print(f"- {num_captured['anchor']} anchor images saved in {ANC_PATH}")
print(f"- {num_captured['positive']} positive images saved in {POS_PATH}")

Getting our imaage directories as datasets


In [ ]:

anchor=tf.data.Dataset.list_files(ANC_PATH+'/*.jpg').take(200)
positive=tf.data.Dataset.list_files(POS_PATH+'/*.jpg').take(200)
negative=tf.data.Dataset.list_files(NEG_PATH+'/*.jpg').take(200)


In [ ]:
# Modify how we create training pairs to ensure equal positive/negative examples
positive_samples = min(len(list(anchor)), len(list(positive)))
negative_samples = positive_samples
anchor = anchor.take(positive_samples)
positive = positive.take(positive_samples)
negative = negative.take(negative_samples)

# **Preparing Data**



## Dataset Creation

Converting our image directories into TensorFlow datasets and creating balanced training pairs:
- Equal numbers of positive and negative pairs
- Proper train/validation splits
- Data augmentation for better generalization

## Image Preprocessing Pipeline

Implementing preprocessing functions to:
- Load and decode images
- Apply data augmentation (flips, brightness, contrast)
- Resize images to consistent dimensions
- Normalize pixel values



In [ ]:
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    # Add basic image augmentation for better model generalization
    img = tf.image.random_flip_left_right(img)  # Random horizontal flip
    img = tf.image.random_brightness(img, 0.2)   # Slight brightness adjustment
    img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
    img = tf.image.resize(img, (100, 100))
    # Handle potential numerical instabilities
    img = tf.cast(img, tf.float32) / 255.0
    return img

Combine both positive and negative pairs into a single dataset with a mix of matching and non-matching pairs


In [ ]:
positives=tf.data.Dataset.zip((anchor,positive,tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives=tf.data.Dataset.zip((anchor,negative,tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))

raw_data = positives.concatenate(negatives) 

Calculate split sizes


In [ ]:
total_size = len(raw_data)
train_size = round(total_size * 0.7)
val_size = total_size - train_size

In [ ]:
val_size

In [ ]:
raw_data

#The first string is our file path to the specific image, second string is the path to either the positive or neggative image
#last value determines whether its +ve or -ve for verification

In [ ]:
sample=raw_data.as_numpy_iterator()

In [ ]:
eg=sample.next()

## Training Data Pipeline

Building an optimized data pipeline with:
- Efficient preprocessing with parallel calls
- Proper batching and caching
- Prefetching for training performance

BUILD, TRAIN AND TEST PARTITION 

In [ ]:
#Function to preprcess the input and validation images as twins
def preprocess_twins(input_img, validation_img, label):
    try:
        print("Input types:", type(input_img), type(validation_img), type(label))
        print("Input image values:", input_img)
        
        def process_single_image(img):
            # If the image is already a tensor, we need to handle it differently
            if isinstance(img, tf.Tensor):
                # If it's already a preprocessed image tensor
                if img.dtype == tf.float32:
                    return tf.image.resize(img,(100,100))
                # If it's a string tensor (filepath)
                elif img.dtype == tf.string:
                    img = tf.io.read_file(img)
                    img = tf.io.decode_jpeg(img, channels=3)
                    img = tf.cast(img, tf.float32) / 255.0
                    img = tf.image.resize(img, (100, 100))
                    return img
            return None
        
        processed_input = process_single_image(input_img)
        processed_validation = process_single_image(validation_img)
        
        return processed_input, processed_validation, label
    except Exception as e:
        print(f"Error processing images: {str(e)}")
        raise

In [ ]:
# Check the first element of the dataset
sample = next(iter(raw_data))
print("Sample structure:", [type(x) for x in sample])
print("First element shape/type:", tf.shape(sample[0]), sample[0].dtype)

In [ ]:
res = preprocess_twins(*eg)  # * collecting the eg values from the register

In [ ]:
res[1]

In [ ]:
res[2]

# **Building Siamese Network**



## Embedding Network Architecture

The embedding network extracts feature vectors from face images with:
- Multiple convolutional blocks with increasing filter sizes
- Batch normalization for training stability
- Dropout layers to prevent overfitting
- Dense output layer producing embedding vectors


In [ ]:
def build_data_pipeline(data, batch_size=32, training=True):
    """
    Creates an optimized data pipeline that properly handles caching.
    The order of operations is crucial for efficient data processing.
    """
    # First preprocess the raw images
    data = data.map(preprocess_twins, num_parallel_calls=tf.data.AUTOTUNE)
    
    if training:
        # For training data, shuffle before batching
        data = data.shuffle(1000)
    
    # Batch the data
    data = data.batch(batch_size, drop_remainder=True)
    
    # Cache after batching to store complete batches
    data = data.cache()
    
    # Prefetch at the end for pipeline efficiency
    data = data.prefetch(tf.data.AUTOTUNE)
    
    return data


Training partition

In [ ]:
# Verify the shapes
def verify_shapes(dataset, name):
    print(f"\nVerifying {name} shapes:")
    for batch in dataset.take(1):
        print(f"Input shape: {batch[0].shape}")
        print(f"Validation shape: {batch[1].shape}")
        print(f"Label shape: {batch[2].shape}")
        break



In [ ]:
# Create and verify datasets with monitoring
def create_and_verify_datasets(raw_data, batch_size=32):
    # Calculate split sizes
    total_size = len(raw_data)
    train_size = round(total_size * 0.7)
    
    # Split raw data
    raw_train = raw_data.take(train_size)
    raw_val = raw_data.skip(train_size)
    
    # Create pipelines
    train_data = build_data_pipeline(raw_train, batch_size, training=True)
    val_data = build_data_pipeline(raw_val, batch_size, training=False)
    
    # Verify both datasets
    print("\nDataset Statistics:")
    print(f"Total examples: {total_size}")
    print(f"Training examples: {train_size}")
    print(f"Validation examples: {total_size - train_size}")
    
    return train_data, val_data



In [ ]:
# Using the improved creation function
train_data, val_data = create_and_verify_datasets(raw_data)

In [ ]:
# Check both datasets
verify_shapes(train_data, "Training Data")
verify_shapes(val_data, "Validation Data")

In [ ]:
train_data

In [ ]:
train_sample = train_data.as_numpy_iterator()

In [ ]:
train_sample =train_sample.next()

In [ ]:
len(train_sample[0])

MODEL ENGINEERING

In [ ]:

#https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf  Siememse networks paper

#Builds embedding layer

def make_embedding():
    inp=Input(shape=(100,100,3), name='input_image')

    #First block 
    c1 = Conv2D(64, (10,10), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(inp)
    c1 = BatchNormalization()(c1)
    m1 = MaxPooling2D(64,(2,2),padding='same')(c1)
    m1 = tf.keras.layers.Dropout(0.3)(m1)

    #Second block
    c2 = Conv2D(128, (3,3), activation='relu')(m1)
    m2 = MaxPooling2D(64,(2,2),padding='same')(c2)
    m2 = tf.keras.layers.Dropout(0.3)(m2)

    #Third block 
    c3 = Conv2D(128, (7,7), activation='relu')(m2)
    m3 = MaxPooling2D(64,(2,2),padding='same')(c3)
    m3 = tf.keras.layers.Dropout(0.3)(m3)

    #Fouth block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)


    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [ ]:
embedding = make_embedding()

In [ ]:
embedding.summary()

## Distance Layer Implementation

Custom layer to compute the L1 (Manhattan) distance between embedding vectors:
- Takes two embedding vectors as input
- Calculates absolute difference between them
- Adds small epsilon for numerical stability


In [ ]:
class L1Dist(Layer):

    #Init method for inheritance
    def __init__(self, **kwargs):
        super().__init__()

    #Des the similarity calculation 
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding) + 1e-6

In [ ]:
l1 = L1Dist()

## Complete Siamese Model

The full Siamese architecture:
- Two input branches sharing the same embedding network
- Distance calculation between embeddings
- Final classification layer to output similarity score


In [ ]:
def make_siamese_model():
    # Anchor image input in the network
    input_image = Input(name='input_img', shape=(100,100,3))

    #Validation image in the network
    validation_image = Input(name='validation_img', shape=(100,100,3))

    #Combine siamese distance components
    siamise_layer = L1Dist()
    siamise_layer._name = 'distance'
    distances = siamise_layer(embedding(input_image), embedding(validation_image))

    #Classification layer
    classifier = Dense(1, activation='sigmoid')(distances)

    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')



In [ ]:
siamese_model = make_siamese_model()

In [ ]:
siamese_model.summary()

# **Training Siamese Network**



## Loss Function and Optimizer

Setting up the training configuration:
- Binary cross-entropy loss for similarity prediction
- Adam optimizer with learning rate scheduling
- Gradient clipping to prevent exploding gradients


In [ ]:
loss_function = tf.losses.BinaryCrossentropy()

initial_learning_rate = 5e-5
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=200,
    decay_rate=0.97
)


In [ ]:
opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr_schedule) #learning rate @ 0.0001 initially then gradually reduces

Make a ckeckpoint 

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [ ]:
# Add before training to check for checkpoint 
import shutil
if os.path.exists(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
os.makedirs(checkpoint_dir, exist_ok=True)

Build train step function

In [ ]:
test_batch = train_data.as_numpy_iterator()

In [ ]:
batch1 = test_batch.next()

In [ ]:
batch1[2]

In [ ]:
@tf.function
def train_step(batch):
    # Get anchor and positive/negative images
    X = batch[:2]
    # Get label
    Y = batch[2]
    
    # Record operations with gradient tape
    with tf.GradientTape() as tape:
        # Forward pass through the model
        yhat = siamese_model(X, training=True)
        # Calculate loss
        loss = loss_function(Y, yhat)
    
    # Calculate gradients
    gradients = tape.gradient(loss, siamese_model.trainable_variables)
    # Clip gradients to prevent exploding gradients
    clipped_gradients = [tf.clip_by_value(g, -1.0, 1.0) for g in gradients]
    # Apply gradients to update model
    opt.apply_gradients(zip(clipped_gradients, siamese_model.trainable_variables))
    
    return loss

## Training Loop Implementation

Custom training loop with:
- Epoch-based training with progress tracking
- Checkpoint saving for model persistence
- Loss monitoring during training

In [ ]:
def train(data, EPOCHS):
    for epoch in range(1, EPOCHS+1): 
        print(f'\nEpoch {epoch}/{EPOCHS}')
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Track losses for this epoch
        epoch_loss = tf.keras.metrics.Mean()
        
        for idx, batch in enumerate(data):
            # Get numerical loss value
            loss = train_step(batch)
            epoch_loss.update_state(loss)
            
            # Update progress bar with actual loss value
            progbar.update(
                idx+1, 
                values=[('loss', float(epoch_loss.result()))]
            )
            
        # Save checkpoints every 10 epochs
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

Train the model

In [ ]:
EPOCHS = 50

In [ ]:
# Check data shape
for batch in train_data.take(1):
    print("Input shape:", batch[0].shape)
    print("Validation shape:", batch[1].shape)
    print("Label shape:", batch[2].shape)
    break

# Check model input shape
print("\nModel input shape:")
for layer in siamese_model.layers:
    if isinstance(layer, tf.keras.layers.InputLayer):
        print(f"{layer.name}: {layer.input_shape}")

In [ ]:
# Initialize model by running one batch through it
for batch in train_data.take(1):
    X = batch[:2]
    # Do a forward pass to initialize variables
    _ = siamese_model(X, training=False)

Train model 

In [ ]:
#train(train_data, EPOCHS)


# **Model Evaluation**

## Performance Metrics

Evaluating the model using:
- Precision: Accuracy of positive predictions
- Recall: Ability to find all positive matches
- F1 Score: Harmonic mean of precision and recall

In [ ]:
from tensorflow.keras.metrics import Precision, Recall


In [ ]:
test_data = raw_data.skip(train_size)
test_data = test_data.map(preprocess_twins)
test_data = test_data.batch(32)
test_data = test_data.prefetch(tf.data.AUTOTUNE)
# Create a balanced test dataset with both positive and negative examples
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))

# Combine and shuffle
balanced_data = positives.concatenate(negatives).shuffle(buffer_size=1000)

# Split into train/test
total_size = len(balanced_data)
train_size = round(total_size * 0.7)

train_data = balanced_data.take(train_size)
test_data = balanced_data.skip(train_size)

# Process for the model
train_data = train_data.map(preprocess_twins).batch(32).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(preprocess_twins).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
def evaluate_model(model, test_data):
    # Initialize metrics
    precision_metric = Precision()
    recall_metric = Recall()
    
    print("Evaluating model on test data...")
    for batch in test_data:
        # Get inputs and labels
        X = batch[:2]
        y = batch[2]
        
        # Generate predictions
        y_pred = model(X, training=False)
        
        # Update metrics
        precision_metric.update_state(y, y_pred)
        recall_metric.update_state(y, y_pred)
    
    # Get metric results
    precision = precision_metric.result().numpy()
    recall = recall_metric.result().numpy()
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print("\n===== Model Evaluation =====")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    
    return precision, recall, f1_score

## Threshold Analysis

Finding the optimal decision threshold:
- Testing different threshold values
- Analyzing precision-recall tradeoffs
- Visualizing threshold impact on performance

In [ ]:
def analyze_thresholds(model, test_data):
    all_preds = []
    all_labels = []
    
    for batch in test_data:
        X = batch[:2]
        y = batch[2].numpy()
        y_pred = model(X, training=False).numpy().flatten()
        
        all_preds.extend(y_pred)
        all_labels.extend(y)
    
    # Test different thresholds
    thresholds = np.arange(0, 0.5, 0.02)
    f1_scores = []
    
    for threshold in thresholds:
        binary_preds = (np.array(all_preds) > threshold).astype(int)
        
        # Calculate metrics
        tp = np.sum((binary_preds == 1) & (np.array(all_labels) == 1))
        fp = np.sum((binary_preds == 1) & (np.array(all_labels) == 0))
        fn = np.sum((binary_preds == 0) & (np.array(all_labels) == 1))
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        f1_scores.append(f1)
    
    # Find best threshold
    best_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_idx]
    
    print(f"Optimal threshold: {best_threshold:.4f} with F1: {f1_scores[best_idx]:.4f}")
    return best_threshold

In [ ]:
def evaluate_with_threshold(model, test_data, threshold):
    all_preds = []
    all_labels = []
    
    for batch in test_data:
        X = batch[:2]
        y = batch[2].numpy()
        y_pred = model(X, training=False).numpy().flatten()
        
        all_preds.extend(y_pred)
        all_labels.extend(y)
    
    # Apply threshold
    binary_preds = (np.array(all_preds) > threshold).astype(int)
    
    # Calculate metrics
    tp = np.sum((binary_preds == 1) & (np.array(all_labels) == 1))
    fp = np.sum((binary_preds == 1) & (np.array(all_labels) == 0))
    fn = np.sum((binary_preds == 0) & (np.array(all_labels) == 1))
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"With threshold {threshold:.4f}:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    return precision, recall, f1

In [ ]:
precision, recall, f1_score = evaluate_model(siamese_model, test_data)


In [ ]:
best_threshold = analyze_thresholds(siamese_model, test_data)


In [ ]:
precision, recall, f1 = evaluate_with_threshold(siamese_model, test_data, best_threshold)


## Results Visualization

Visualizing model predictions:
- Displaying image pairs with prediction results
- Showing distribution of similarity scores
- Highlighting the decision boundary


In [ ]:
def visualize_results(model, test_data, threshold=0.5, num_examples=5):
    """
    Visualize prediction results from the Siamese model
    
    Args:
        model: The trained Siamese model
        test_data: The test dataset
        threshold: Decision threshold for positive/negative classification
        num_examples: Number of examples to visualize
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Get a batch of data
    test_batch = next(test_data.as_numpy_iterator())
    X = test_batch[:2]  # Input images
    y_true = test_batch[2]  # True labels
    
    # Generate predictions
    y_pred = model(X, training=False).numpy().flatten()
    
    # Limit to the number of examples requested
    n = min(num_examples, len(y_true))
    
    for i in range(n):
        # Set up the figure
        fig, axes = plt.subplots(1, 2, figsize=(10, 4))
        
        # Display the image pair
        axes[0].imshow(X[0][i])
        axes[0].set_title('Anchor Image')
        axes[0].axis('off')
        
        axes[1].imshow(X[1][i])
        
        # Determine match status and correctness
        is_match = y_true[i] > 0.5
        predicted_match = y_pred[i] > threshold
        is_correct = (is_match == predicted_match)
        
        # Create detailed title
        title = f"{'Match' if is_match else 'No Match'} (Ground Truth)\n"
        title += f"Prediction: {'Match' if predicted_match else 'No Match'}\n"
        title += f"Confidence: {y_pred[i]:.4f}, Threshold: {threshold:.2f}"
        
        # Set the title color based on match/no-match, not correctness
        color = 'green' if is_match else 'red'
        
        axes[1].set_title(title, color=color)
        axes[1].axis('off')
        
        
        plt.tight_layout()
        plt.subplots_adjust(bottom=0.15)  # Make room for the text
        plt.show()

In [ ]:
visualize_results(siamese_model, test_data, threshold=best_threshold)


In [ ]:
def visualize_threshold_analysis(model, test_data):
    """
    Visualize the impact of different thresholds on model performance
    
    Args:
        model: The trained Siamese model
        test_data: The test dataset
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Collect all predictions
    all_preds = []
    all_labels = []
    
    for batch in test_data:
        X = batch[:2]
        y = batch[2].numpy()
        y_pred = model(X, training=False).numpy().flatten()
        
        all_preds.extend(y_pred)
        all_labels.extend(y)
    
    # Convert to numpy arrays
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    # Compute metrics at different thresholds
    thresholds = np.arange(0, 1.01, 0.05)
    precisions = []
    recalls = []
    f1_scores = []
    
    for threshold in thresholds:
        binary_preds = (all_preds > threshold).astype(int)
        
        # Calculate metrics
        tp = np.sum((binary_preds == 1) & (all_labels == 1))
        fp = np.sum((binary_preds == 1) & (all_labels == 0))
        fn = np.sum((binary_preds == 0) & (all_labels == 1))
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
    
    # Find optimal threshold
    best_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_idx]
    
    # Plot metrics vs threshold
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, precisions, 'b-', label='Precision')
    plt.plot(thresholds, recalls, 'g-', label='Recall')
    plt.plot(thresholds, f1_scores, 'r-', label='F1 Score')
    
    # Mark the optimal threshold
    plt.axvline(x=best_threshold, color='k', linestyle='--', 
                label=f'Best Threshold = {best_threshold:.2f}')
    
    plt.title('Model Performance vs. Threshold')
    plt.xlabel('Threshold')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    # Plot prediction distribution
    plt.figure(figsize=(10, 6))
    
    # Separate predictions for positive and negative pairs
    pos_preds = all_preds[all_labels == 1]
    neg_preds = all_preds[all_labels == 0]
    
    # Plot histograms
    plt.hist(pos_preds, bins=20, alpha=0.6, color='green', label='Matching Pairs')
    plt.hist(neg_preds, bins=20, alpha=0.6, color='red', label='Non-matching Pairs')
    
    plt.axvline(x=best_threshold, color='k', linestyle='--', 
                label=f'Best Threshold = {best_threshold:.2f}')
    
    plt.title('Distribution of Similarity Scores')
    plt.xlabel('Model Prediction (Similarity Score)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
visualize_threshold_analysis(siamese_model, test_data)


# **Model Saving and Deployment**


## Model Persistence

Saving the trained model with:
- Full model architecture and weights
- Embedding model for feature extraction
- Metadata including optimal threshold
- Usage examples for deployment

In [ ]:
def save_model(model, embedding_model, best_threshold, model_metrics, version=None):
    """
    Save the Siamese model with metadata
    
    Args:
        model: The trained Siamese model
        embedding_model: The embedding part of the model
        best_threshold: The optimal threshold for classification
        model_metrics: Dictionary with precision, recall, and f1_score
        version: Optional version string (defaults to timestamp)
    """
    import os
    import time
    import json
    import tensorflow as tf
    
    # Create directories
    base_dir = 'models'
    os.makedirs(base_dir, exist_ok=True)
    
    # Create version name with timestamp if not provided
    if not version:
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        version = f"siamese_model_{timestamp}"
    
    # Create model directory
    model_dir = os.path.join(base_dir, version)
    os.makedirs(model_dir, exist_ok=True)
    
    # 1. Save full model
    full_model_path = os.path.join(model_dir, 'full_model')
    model.save(full_model_path)
    print(f"Full model saved to: {full_model_path}")
    
    # 2. Save embedding model
    embedding_path = os.path.join(model_dir, 'embedding')
    embedding_model.save(embedding_path)
    print(f"Embedding model saved to: {embedding_path}")
    
    # 3. Save model weights
    weights_path = os.path.join(model_dir, 'model_weights.h5')
    model.save_weights(weights_path)
    print(f"Model weights saved to: {weights_path}")
    
    # 4. Save model metadata
    metadata = {
        'model_name': version,
        'date_created': time.strftime("%Y-%m-%d %H:%M:%S"),
        'framework_version': tf.__version__,
        'image_size': [100, 100, 3],
        'metrics': {
            'precision': float(model_metrics['precision']),
            'recall': float(model_metrics['recall']),
            'f1_score': float(model_metrics['f1_score'])
        },
        'optimal_threshold': float(best_threshold),
    }
    
    metadata_path = os.path.join(model_dir, 'model_info.json')
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"Model metadata saved to: {metadata_path}")
    
    # 5. Create usage example file
    example_code = f"""# Example code to use the Siamese face verification model

import tensorflow as tf
import cv2
import numpy as np

# Define L1Dist layer (needed for model loading)
class L1Dist(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
    
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

# Load the saved model
model_path = '{full_model_path}'
model = tf.keras.models.load_model(
    model_path, 
    custom_objects={{'L1Dist': L1Dist}}
)

def preprocess_image(img_path):
    # Read image
    img = cv2.imread(img_path)
    # Resize to model input size
    img = cv2.resize(img, (100, 100))
    # Normalize pixel values
    img = img / 255.0
    return img

def verify_faces(img_path1, img_path2, threshold={best_threshold}):
    # Preprocess images
    img1 = preprocess_image(img_path1)
    img2 = preprocess_image(img_path2)
    
    # Add batch dimension
    img1 = np.expand_dims(img1, axis=0)
    img2 = np.expand_dims(img2, axis=0)
    
    # Get prediction
    result = model.predict([img1, img2])
    similarity_score = result[0][0]
    
    # Determine if match based on optimal threshold
    is_match = similarity_score > threshold
    
    return {{'is_match': bool(is_match), 'score': float(similarity_score)}}

# Example usage
# result = verify_faces('person1.jpg', 'person2.jpg')
# print(f"Match: {{result['is_match']}}, Score: {{result['score']:.4f}}")
"""
    
    example_path = os.path.join(model_dir, 'usage_example.py')
    with open(example_path, 'w') as f:
        f.write(example_code)
    print(f"Usage example saved to: {example_path}")
    
    print(f"\nModel successfully saved to {model_dir}")
    return model_dir


# Example usage:
# model_metrics = {
#     'precision': precision,
#     'recall': recall, 
#     'f1_score': f1_score
# }
# 
# saved_dir = save_model(
#     model=siamese_model,
#     embedding_model=embedding,
#     best_threshold=best_threshold,
#     model_metrics=model_metrics
# )

In [ ]:
model_metrics = {
    'precision': precision,
    'recall': recall, 
    'f1_score': f1
}

saved_dir = save_model(
    model=siamese_model,
    embedding_model=embedding,
    best_threshold=best_threshold,
    model_metrics=model_metrics
)